In [3]:
import os

os.chdir("../")

In [9]:
%pwd

'd:\\Project\\ETE_Projects\\image_classifer\\Mushroom_Edibility_Classifier'

In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    training_data : Path
    validation_data : Path
    test_data : Path



@dataclass
class PrepareCallbackConfig:
    root_dir: Path
    checkpoint_dir: Path 
    tensorboard_dir: Path 
    

In [11]:
from Mushroom_edibility_classifier.constants import *
from Mushroom_edibility_classifier.utils.common import read_yaml, create_directories


In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def PrepareCallbackConfig(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_checkpoint_dir = os.path.dirname(config.checkpoint_path)
        tensorboard_dir = Path(config.tensorboard_dir)
        root_dir = Path(self.config.artifacts_root)

        create_directories([model_checkpoint_dir, tensorboard_dir])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(root_dir),
            checkpoint_dir=Path(model_checkpoint_dir),
            tensorboard_dir=Path(tensorboard_dir)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        
        # Paths to pre-split datasets
        base_data_dir = Path(self.config.data_ingestion.unzip_dir)
        training_data = base_data_dir / "splitted_dataset/train"  # Changed to train folder
        validation_data = base_data_dir / "splitted_dataset/val"   # Added validation path
        test_data = base_data_dir / "splitted_dataset/test"       # Added test path (optional)

        # Create required directories
        create_directories([
            Path(training.root_dir),
            validation_data,  # Ensure val directory exists
            test_data         # Ensure test directory exists (if used)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=training_data,
            validation_data=validation_data,  # Added
            test_data=test_data,              # Added (optional)
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
self.config.training

NameError: name 'self' is not defined

In [13]:
import time
import tensorflow as tf

In [14]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(self.config.checkpoint_dir,"model.keras"),
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [15]:
import os
import urllib.request as request
from zipfile import ZipFile
import time
import tensorflow as tf
from pathlib import Path
from typing import List, Optional

[2025-06-06 20:54:16,882: WARNING: module_wrapper: From C:\Users\ramsp\AppData\Local\Temp\ipykernel_1088\2827125363.py:1: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.
]
Eager execution enabled: True


In [ ]:
tf.compat.v1.enable_eager_execution()
print("Eager execution enabled:", tf.executing_eagerly())

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        tf.config.run_functions_eagerly(True) 
    
    def get_base_model(self):
        """Load the base model for training"""
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def create_data_generators(self):
        """Create data generators for pre-split train/val/test datasets"""
        # Common preprocessing parameters
        datagen_kwargs = dict(
            rescale=1./255,
        )

        # Common flow parameters
        flow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode='categorical'
        )

        # Validation generator (no augmentation)
        val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
        self.val_generator = val_datagen.flow_from_directory(
            directory=self.config.validation_data,  # Path to validation folder
            shuffle=False,
            **flow_kwargs
        )

        # Training generator (with optional augmentation)
        if self.config.params_is_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagen_kwargs
            )
        else:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)

        self.train_generator = train_datagen.flow_from_directory(
            directory=self.config.training_data,  # Path to train folder
            shuffle=True,
            **flow_kwargs
        )

        # Optional test generator
        if hasattr(self.config, 'test_data'):
            test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
            self.test_generator = test_datagen.flow_from_directory(
                directory=self.config.test_data,
                shuffle=False,
                **flow_kwargs
            )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """Save the trained model"""
        model.save(path)

    def train(self, callback_list: Optional[List[tf.keras.callbacks.Callback]] = None):
        """Train the model with callbacks"""
        if callback_list is None:
            callback_list = []

        # Calculate steps
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.val_generator.samples // self.val_generator.batch_size
        self.model.optimizer = tf.keras.optimizers.Adam()

        # Train the model
        history = self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.val_generator,
            validation_steps=self.validation_steps,
            callbacks=callback_list
        )

        # Save and return training history
        self.save_model(self.config.trained_model_path, self.model)
        



[2025-06-06 22:37:59,432: WARNING: module_wrapper: From C:\Users\ramsp\AppData\Local\Temp\ipykernel_20816\130072089.py:1: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.
]
Eager execution enabled: True


In [ ]:
config.test_data

AttributeError: 'ConfigurationManager' object has no attribute 'test_data'

In [ ]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.PrepareCallbackConfig()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.create_data_generators()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-06-06 22:38:11,631: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-06 22:38:11,640: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-06 22:38:11,643: INFO: common: created directory at: artifacts]
[2025-06-06 22:38:11,645: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2025-06-06 22:38:11,646: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_logs]
[2025-06-06 22:38:11,647: INFO: common: created directory at: artifacts\training]
[2025-06-06 22:38:11,648: INFO: common: created directory at: artifacts\data_ingestion\splitted_dataset\val]
[2025-06-06 22:38:11,648: INFO: common: created directory at: artifacts\data_ingestion\splitted_dataset\test]
[2025-06-06 22:38:12,055: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 282 images belonging to 2 clas

d:\Project\ETE_Projects\image_classifer\Mushroom_Edibility_Classifier\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/60


d:\Project\ETE_Projects\image_classifer\Mushroom_Edibility_Classifier\venv\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


203/282 ━━━━━━━━━━━━━━━━━━━━ 6:29 5s/step - accuracy: 0.6107 - loss: 1.1120

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[]


In [ ]:
!nvidia-smi

Fri Jun  6 22:05:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.52                 Driver Version: 576.52         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   58C    P8              4W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----